In [1]:
import matplotlib.pyplot as plt

from mesa import Agent, Model

from mesa.batchrunner import BatchRunner
from mesa.datacollection import DataCollector
from mesa.time import RandomActivation
from mesa.space import MultiGrid

import numpy as np
import random

%matplotlib inline

# Creating the models
This section creates the models for the agent and the environment the agents will work in.

The agents are given a random number for age and what age they're given will predict what probability of survival they will have. This could be more efficiently and effectively made into a function that can calculate the curve for probability. Movement is random.

Other agent factors may be taken into account: Health, PPE, environment, distancing, and geneology.

The environment modelling is basic, it does not take into account for barriers and agents can occupy the same space. The grid also repeats, when an agent tries to go over the edge it reappears at the opposite side of the grid, there is no escape.

Once an agent is declared dead then they stop moving and are considered as no longer 'infectious'. Though disposal of infected bodies can be a whole different issue.

In [2]:
def average_age(model):
    """
    Calculate the average age of the population.
    This will be displayed in the linegraph just below the grid.
    """
    agent_age = []
    for agent in model.schedule.agents:
        if agent.health > 0:
            agent_age.append(agent.age)
    if len(agent_age) > 0:
        return sum(agent_age)/len(agent_age)
    else:
        return None
class InfectedAgent(Agent):
    """
    Initializing an agent with health state '2' and random age.
    If there were going to be more agent specific variables, 
    like the fitness, diet, PPE, or anything other, this is where
    I would declare a function for it.
    
    Health has 4 states: 
        0 - Dead; 
        1 - infected; 
        2 - healthy (no anti-bodies); 
        3 - healthy (anti-bodies)

    """
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.health = 2
        
        # Age is a big role in survival, as people age their immune system weakens
        # These are percentage values of survival probability.
        self.age = random.randint(1, 86)
        if self.age < 40:
            self.survival = 99
        elif self.age <= 60:
            self.survival = 90
        elif self.age > 60:
            self.survival = 15
        self.infect_day_count = 0

    def move(self):
        # 2 ways to get neighbors: 
        #     Moore - 8 surrounding cells; 
        #     Von Neumann - Only XY axis.
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore = True,
            include_center = False,
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)
            
    def infect(self):
        """Spreading the infect to cellmates"""
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            for cellmate in cellmates:
                if self.health == 1 and cellmate.health == 2: 
                    cellmate.health = self.health
                elif self.health == 2 and cellmate.health == 1:
                    self.health = cellmate.health
                    self.immunity()
                elif self.health == 1 and cellmate.health == 1:
                    self.immunity()
    
    def immunity(self):
        if self.health == 1:            
            # Random roll of fate whether one survives or not.
            # Probability of survival helps.
            if self.survival < random.randint(1, 100):
                self.health = 0
                
            # Survive for 14 days and you're considered 'immmune'.
            self.infect_day_count += 1
            if self.infect_day_count == 14:
                self.health = 3        
        
    def step(self):
        # Make the agent move in environment by 1 step.
        if self.health != 0:
            self.move()
        # Spreading (1) or catching (2) the infection.
        if self.health == 1 or self.health == 2:
            self.infect()

class InfectionModel(Model):
    """
    Declaring the model environment with some number of agents.
    On step, instructions might be given to define obstacles in the
    environment though there might be a better way of modelling the 
    environment.
    """
    def __init__(self, N, width, height):
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.running = True

        # Create agents
        for i in range(self.num_agents):
            a = InfectedAgent(i, self)
            self.schedule.add(a)

            # Assign the agents to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
        
        self.datacollector = DataCollector(
            model_reporters={"Average Age": average_age},
            agent_reporters={"Health": "health"},
        )
        
        # Infect a random agent as Patient Zero.
        pz = random.choice(list(self.schedule.agents))
        pz.health = 1
                
    def step(self):
        '''Advance the model by one step'''
        self.datacollector.collect(self)
        self.schedule.step()

Now to create the visualization using the provided packages and server.

The agent portrayal defines how the dots will be viewed depending on the health state of the agent.

The histogram object renders the *Average Age* function above.

The visual objects are called into existence with their appropriate variables and sent to the server to be rendered.

The grid rendered is 10 x 10 width by height with "N"=100 agents. (Server currently had trouble simulating more than 10 x 10 grids.)

In [3]:
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.visualization.ModularVisualization import ModularServer, VisualizationElement

In [4]:
def agent_portrayal(agent):
    """
    How the agents are portrayed depending on what health state they are.
    0 - Mortis/dead: Black Dot
    1 - Infected: Red Dot
    2 - Healthy (not immune): Yellow Dot
    3 - Healthy (immune): Green Dot
    """
    portrayal = {
        "Shape": "circle",
        "Filled": "true",
    }
    if agent.health == 0:
        portrayal["Color"] = "black"
        portrayal["Layer"] = 3
        portrayal["r"] = 0.25
        
    elif agent.health == 1:
        portrayal["Color"] = "red"
        portrayal["Layer"] = 2
        portrayal["r"] = 0.5
        
    elif agent.health == 2:
        portrayal["Color"] = "yellow"
        portrayal["Layer"] = 1
        portrayal["r"] = 0.75
        
    elif agent.health == 3:
        portrayal["Color"] = "green"
        portrayal["Layer"] = 0
        portrayal["r"] = 1.0
    return portrayal

In [5]:

class HistogramModuleHealth(VisualizationElement):
    """Creating a custom visualization"""
    package_includes = ["Chart.min.js"]
    local_includes = ["HistogramModule.js"]

    def __init__(self, bins, canvas_height, canvas_width):
        self.canvas_height = canvas_height
        self.canvas_width = canvas_width
        self.bins = bins
        new_element = "new HistogramModule({}, {}, {})"
        new_element = new_element.format(bins,
                                         canvas_width,
                                         canvas_height)
        self.js_code = "elements.push(" + new_element + ");"

    def render(self, model):
        health_vals = [agent.health for agent in model.schedule.agents]
        hist = np.histogram(health_vals, bins=self.bins)[0]
        return [int(x) for x in hist]

In [6]:
grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)

chart = ChartModule([{"Label": "Average Age",
                      "Color": "Black"}],
                    data_collector_name='datacollector')

histogram = HistogramModuleHealth(list(range(10)), 200, 500)

server = ModularServer(InfectionModel,
                       [grid, chart, histogram],
                       "Infection Model",
                       {"N":10, "width":10, "height":10})

# server.port = 8521 # This is the default value
server.launch()

Interface starting at http://127.0.0.1:8521


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type":"get_step","step":31}
{"type":"get_step","step":32}
{"type":"get_step